# Building a ReAct Agent with Real Tools

In this notebook, we'll build a **ReAct agent** that can autonomously reason about problems and choose from multiple real tools to solve them.

**ReAct** (Reasoning + Acting) is simple but powerful:
1. **Reason**: The LLM explains what it's thinking
2. **Act**: The LLM calls a tool
3. **Observe**: The tool returns results
4. **Repeat**: Until the task is complete

## The Tools We'll Use

- **MCP Calendar Server**: Manage events (the 9 calendar tools you used before)
- **RAG System**: Search through documents for information
- **Calculator**: Simple arithmetic

The agent will autonomously decide which tools to use and in what order based on the user request!

## Setup: Install Dependencies

In [ ]:
!pip3 install -q llama-stack-client==0.3.0 mcp httpx

Define Llama Stack endpoint and Calendar MCP Server endpoint.

In [ ]:
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger
from mcp import ClientSession
from mcp.client.sse import sse_client
import json
from typing import Dict, Any, List
import re
import logging

# Reduce noise from HTTP logs
logging.getLogger("httpx").setLevel(logging.WARNING)

# Connect to Llama Stack Server
base_url = "http://llama-stack-service:8321"
client = LlamaStackClient(base_url=base_url)
model = "llama32"

# MCP Server URL
mcp_server_url = "http://canopy-mcp-calendar-mcp-server:8080/sse"

print(f"✅ Connected to Llama Stack at: {base_url}")
print(f"✅ Using model: {model}")
print(f"✅ MCP Server: {mcp_server_url}")

## Setting Up Our Tools

Let's create wrapper functions for our three tool categories: Calculator, MCP Calendar, and RAG.

### Tool 1: Calculator

In [ ]:
def calculator_tool(operation: str, a: float, b: float) -> str:
    """Performs basic arithmetic operations."""
    try:
        if operation == "add":
            result = a + b
        elif operation == "subtract":
            result = a - b
        elif operation == "multiply":
            result = a * b
        elif operation == "divide":
            if b == 0:
                return "Error: Cannot divide by zero"
            result = a / b
        else:
            return f"Error: Unknown operation '{operation}'"
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Calculator tool ready")

### Tool 2: MCP Calendar Tools

In [ ]:
async def call_mcp_tool(tool_name: str, arguments: Dict[str, Any]) -> str:
    """Call an MCP calendar tool and return the result as a string."""
    try:
        async with sse_client(mcp_server_url) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                result = await session.call_tool(tool_name, arguments=arguments)
                return "\n".join([content.text for content in result.content])
    except Exception as e:
        return f"Error calling MCP tool: {str(e)}"

async def get_mcp_tools_description() -> str:
    """Get descriptions of all available MCP tools."""
    async with sse_client(mcp_server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await session.list_tools()
            
            descriptions = []
            for tool in tools.tools:
                descriptions.append(f"  - mcp_{tool.name}: {tool.description}")
            
            return "\n".join(descriptions)

# Get and display available MCP tools
mcp_tools_desc = await get_mcp_tools_description()
print("✅ MCP Calendar tools available:")
print(mcp_tools_desc)

### Tool 3: RAG (Document Search)

Let's set up the RAG system so our agent can search through documents.

In [ ]:
# Check if we have a vector store already, if not create one
try:
    # List existing vector stores
    vector_stores = client.vector_stores.list()
    
    if vector_stores.data and len(vector_stores.data) > 0:
        # Use the first available vector store
        vector_store = vector_stores.data[0]
        print(f"✅ Using existing vector store: {vector_store.id}")
    else:
        raise Exception("No vector store exists")
except Exception as e:
    print(f"⚠️  Could not set up vector store: {e}")
    vector_store = None

In [ ]:
def rag_search_tool(query: str, max_results: int = 3) -> str:
    """Search through documents using RAG and return relevant information."""
    if not vector_store:
        return "Error: RAG system not initialized. Please set up a vector store first."
    
    try:
        # Search the vector database
        search_results = client.vector_stores.search(
            vector_store_id=vector_store.id,
            query=query,
            max_num_results=max_results,
            search_mode="vector"
        )
        
        if not search_results.data:
            return "No relevant documents found."
        
        # Format the results
        results = []
        for i, result in enumerate(search_results.data, 1):
            content = result.content if hasattr(result, 'content') else str(result)
            results.append(f"Result {i}: {content}")
        
        return "\n\n".join(results)
    except Exception as e:
        return f"Error performing RAG search: {str(e)}"

print("✅ RAG search tool ready")

## Building the ReAct Agent

Now let's build the ReAct agent that can autonomously choose which tool to use!

In [ ]:
def create_react_system_prompt() -> str:
    """Create a system prompt for the ReAct agent."""
    
    return f"""You are a helpful assistant that can use tools to answer questions.

Available tools:

1. calculator
   - Performs arithmetic operations (add, subtract, multiply, divide)
   - Input format: {{"operation": "add"|"subtract"|"multiply"|"divide", "a": number, "b": number}}

2. MCP Calendar Tools:
{mcp_tools_desc}
   - For calendar tools, use format: {{"tool_name": "mcp_TOOLNAME", "arguments": {{...}}}}

3. rag_search
   - Searches through knowledge base documents for information
   - Input format: {{"query": "your search query", "max_results": 3}}
   - Use this when the user asks about concepts, definitions, or information not in the calendar

Use this format for your responses:

Thought: [Explain your reasoning about what to do next]
Action: [tool_name]
Action Input: {{"key": "value"}}

After receiving an observation, you can either:
1. Continue with another Thought/Action/Action Input if you need more information
2. Provide a final answer with: Final Answer: [your answer]

Important:
- Always explain your thinking in the Thought section
- Choose the most appropriate tool for the task
- For calendar operations, use the MCP tools (mcp_get_upcoming_events, mcp_create_event, etc.)
- For knowledge questions, use rag_search
- For calculations, use calculator
"""

print("✅ ReAct system prompt created")

## The ReAct Loop Implementation

In [ ]:
def parse_react_response(response: str) -> Dict[str, Any]:
    """Parse the LLM's ReAct-formatted response."""
    result = {}
    
    # Extract Thought
    thought_match = re.search(r'Thought:\s*(.+?)(?=\n(?:Action|Final Answer):|$)', response, re.DOTALL)
    if thought_match:
        result['thought'] = thought_match.group(1).strip()
    
    # Check for Final Answer
    final_answer_match = re.search(r'Final Answer:\s*(.+)', response, re.DOTALL)
    if final_answer_match:
        result['final_answer'] = final_answer_match.group(1).strip()
        return result
    
    # Extract Action
    action_match = re.search(r'Action:\s*(.+?)(?=\n|$)', response)
    if action_match:
        result['action'] = action_match.group(1).strip()
    
    # Extract Action Input
    action_input_match = re.search(r'Action Input:\s*({.+})', response, re.DOTALL)
    if action_input_match:
        try:
            json_str = action_input_match.group(1).strip()
            # Clean up common JSON formatting issues
            json_str = re.sub(r'\n\s*', ' ', json_str)  # Remove newlines
            result['action_input'] = json.loads(json_str)
        except json.JSONDecodeError as e:
            result['error'] = f"Failed to parse action input: {e}"
    
    return result


async def execute_tool(action: str, action_input: Dict[str, Any]) -> str:
    """Execute the appropriate tool based on the action."""
    
    if action == "calculator":
        return calculator_tool(
            action_input.get("operation"),
            action_input.get("a"),
            action_input.get("b")
        )
    
    elif action.startswith("mcp_"):
        # MCP calendar tool
        tool_name = action[4:]  # Remove 'mcp_' prefix
        arguments = action_input.get("arguments", action_input)
        return await call_mcp_tool(tool_name, arguments)
    
    elif action == "rag_search":
        return rag_search_tool(
            action_input.get("query"),
            action_input.get("max_results", 3)
        )
    
    else:
        return f"Error: Unknown tool '{action}'"


async def run_react_agent(user_question: str, max_iterations: int = 5, verbose: bool = True):
    """Run the ReAct agent loop."""
    
    if verbose:
        print("="*70)
        print(f"USER QUESTION: {user_question}")
        print("="*70)
    
    system_prompt = create_react_system_prompt()
    conversation_history = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question}
    ]
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"\n{'='*70}")
            print(f"ITERATION {iteration + 1}")
            print("="*70)
        
        # Get LLM response
        response = client.chat.completions.create(
            model=model,
            messages=conversation_history,
            stream=False,
        )
        
        llm_output = response.choices[0].message.content
        
        if verbose:
            print(f"\n{llm_output}")
        
        # Parse the response
        parsed = parse_react_response(llm_output)
        
        # Check if we have a final answer
        if 'final_answer' in parsed:
            if verbose:
                print("\n" + "="*70)
                print("FINAL ANSWER:")
                print("="*70)
                print(parsed['final_answer'])
                print("="*70)
            return parsed['final_answer']
        
        # Execute the action if we have one
        if 'action' in parsed and 'action_input' in parsed:
            action = parsed['action']
            action_input = parsed['action_input']
            
            if verbose:
                print(f"\n🔧 Executing: {action}")
                print(f"📥 Input: {json.dumps(action_input, indent=2)}")
            
            # Execute the tool
            tool_result = await execute_tool(action, action_input)
            
            if verbose:
                print(f"\n📊 Observation:")
                result_preview = tool_result[:300] + "..." if len(tool_result) > 300 else tool_result
                print(result_preview)
            
            # Add observation to conversation
            observation_text = f"Observation: {tool_result}"
            conversation_history.append({"role": "assistant", "content": llm_output})
            conversation_history.append({"role": "user", "content": observation_text})
        else:
            # LLM didn't follow format properly
            error_msg = "Please follow the Thought/Action/Action Input format or provide a Final Answer."
            if verbose:
                print(f"\n⚠️  {error_msg}")
            conversation_history.append({"role": "assistant", "content": llm_output})
            conversation_history.append({"role": "user", "content": error_msg})
    
    if verbose:
        print("\n" + "="*70)
        print("⚠️  WARNING: Max iterations reached without final answer")
        print("="*70)
    
    return "I couldn't find an answer within the iteration limit."

print("✅ ReAct agent is ready!")

## Let's Test the Agent!

Now let's see the agent in action. Watch how it autonomously chooses the right tool(s) for each task.

### Example 1: Using the Calculator

The agent should recognize this needs a calculator.

In [ ]:
await run_react_agent("What is 847 multiplied by 923?")

### Example 2: Using MCP Calendar

The agent should use the calendar MCP tools.

In [ ]:
await run_react_agent("What events do I have coming up in the next 7 days?")

### Example 3: Using RAG Search

The agent can use RAG to search documents, this is also referred to as Agentic RAG.  

**Note**: If this doesn't work, make sure that you have ingested the Canopy document into the RAG database in your canopy project by running through `5-rag/2-intro-to-RAG.ipynb`.

In [ ]:
await run_react_agent("What is the structure of Canopy in botany?")

### Example 4: Multi-step Reasoning

This requires the agent to:
1. Search calendar for events
2. Perform a calculation

In [ ]:
await run_react_agent("How many events do I have in the next week, multiplied by 2?")

### Example 5: Difficult Requests

Here is a couple of more difficult requests for the agent.  
Run them and see if it manages to do them, and if not, why do you think it fails?

In [ ]:
await run_react_agent("Tell me about my schedule and what Canopy means")

In [ ]:
await run_react_agent("Create a meeting for tomorrow at 2pm called 'Team Sync'")

## Real-World Implications

Agentic seems fantastic, and it is, but there are drawbacks to be aware of when using agentic workflows.

- **Cost**: Each iteration = 1 LLM call = $$$
- **Latency**: More reasoning = slower responses
- **Reliability**: Since there is more context, agents can easier make misstakes, especially if there are a lot of tools
- **Debugging**: Multi-step reasoning is hard to debug

## Try Your Own!

Now it's your turn. Try questions that:
- Require multiple different tools
- Need reasoning about which tool to use
- Combine calendar, knowledge, and computation

See how well the agent handles them!

In [ ]:
# Your turn! Try your own questions:
await run_react_agent("YOUR QUESTION HERE")